In [1]:
!pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to c:\users\divya\appdata\local\temp\pip-req-build-bsl3knqf
  Resolved https://github.com/huggingface/transformers to commit 12c39e5693f7223be162a1e84de026a6545029eb
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached huggingface_hub-0.22.2-py3-none-any.whl.metadata (12 kB)
  Using cached PyYAML-6.0.1-cp311-cp311-win_amd64.whl.metadata (2.1 kB)
  Using cached tokenizers-0.19.1-cp311-none-win_amd64.whl.metadata (6.9 kB)
  Using cached safetensors-0.4.3-cp311-none-win_amd64.whl.metadata (3.9 kB)
  Using cached tqdm-4.66.2-py3-none-any.whl.meta

  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers 'C:\Users\divya\AppData\Local\Temp\pip-req-build-bsl3knqf'


In [48]:
import os
import numpy as np
import pandas as pd
import librosa
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments


In [49]:
# data_dir="C:/Users/divya/OneDrive/Documents/UTAUSTIN/Semester 6/460j/final_project_data/fma_small"
# file_path="C:/Users/divya/OneDrive/Documents/UTAUSTIN/Semester 6/460j/final_project_data/fma_small/000/000002.mp3"
train_path = 'C:/Users/divya/OneDrive/Documents/UTAUSTIN/Semester 6/460j/460jFinalProject/extracted_features_combined.csv'


In [50]:
data_df=pd.read_csv(train_path)

In [51]:
display(data_df.head())

,track_id,0,1,2,3,4,5,6,7,8,...,1176,1177,1178,1179,1180,1181,1182,1183,1184,genre
0,2,-122.713936,117.760094,-42.334183,38.061005,-23.012323,23.920383,-11.944865,13.687291,-7.294582,...,0.000032,0.000033,0.000039,0.000048,0.000078,0.000110,0.000089,0.000050,0.000029,Hip-Hop
1,5,-162.232422,131.435989,-18.426781,51.631371,-15.444844,23.860094,-7.867664,10.686234,-0.314591,...,0.000039,0.000041,0.000060,0.000092,0.000102,0.000146,0.000153,0.000072,0.000038,Hip-Hop
2,10,-75.631538,154.537613,-63.849487,22.506426,2.425359,10.459981,-13.007332,9.818939,-7.640783,...,0.000056,0.000066,0.000069,0.000083,0.000095,0.000103,0.000103,0.000072,0.000058,Pop
3,140,-322.997864,133.684906,15.497437,54.755615,8.718211,37.022747,-2.116411,15.265619,6.175477,...,0.000041,0.000035,0.000031,0.000038,0.000050,0.000052,0.000034,0.000024,0.000022,Folk
4,141,-253.314560,157.769379,-6.081675,26.920168,6.813843,15.842272,-12.930041,10.373278,-0.609945,...,0.000029,0.000025,0.000026,0.000048,0.000095,0.000081,0.000036,0.000029,0.000023,Folk


In [52]:
X = StandardScaler().fit_transform(data_df.drop('genre', axis=1))
y = pd.factorize(data_df['genre'])[0]  # Convert genre to numerical labels


In [53]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [54]:
# Define a dataset class
class GenreDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {'input_ids': self.features[idx], 'labels': self.labels[idx]}

In [55]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
train_dataset = GenreDataset(X_train, y_train)
test_dataset = GenreDataset(X_test, y_test)

In [78]:
class TransformerBlock(nn.Module):
    def __init__(self, embed_size, heads, dropout, forward_expansion):
        super(TransformerBlock, self).__init__()
        self.attention = nn.MultiheadAttention(embed_dim=embed_size, num_heads=heads, dropout=dropout)
        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size)

        self.feed_forward = nn.Sequential(
            nn.Linear(embed_size, forward_expansion * embed_size),
            nn.ReLU(),
            nn.Linear(forward_expansion * embed_size, embed_size),
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, value, key, query, mask):
        attention = self.attention(query, key, value, attn_mask=mask)[0]
        x = self.dropout(self.norm1(attention + query))
        forward = self.feed_forward(x)
        out = self.dropout(self.norm2(forward + x))
        return out

class CustomBERT(nn.Module):
    def __init__(self, embed_size, num_layers, heads, device, forward_expansion, dropout, num_classes):
        super(CustomBERT, self).__init__()
        self.embed_size = embed_size
        self.device = device
        self.layers = nn.ModuleList(
            [TransformerBlock(embed_size, heads, dropout, forward_expansion) for _ in range(num_layers)]
        )

        self.dropout = nn.Dropout(dropout)
        self.fc_out = nn.Linear(embed_size, 8)
        self.sig = nn.Sigmoid()

    def forward(self, x, mask):
        out = x
        for layer in self.layers:
            out = layer(out, out, out, mask)
        out = out.mean(dim=1)
        out = self.dropout(out)
        out = self.fc_out(out)
        return self.sig(out)


In [79]:
# Constants
embed_size = 1186  # Size of each embedding vector
num_layers = 8    # Number of transformer blocks
heads = 2         # Number of heads in multi-head attention mechanism
device = "cuda" if torch.cuda.is_available() else "cpu"
forward_expansion = 4
dropout = 0.1
num_classes = 8   # Assuming 8 genre classes
max_len = 1186    # Maximum length of the input


model = CustomBERT(embed_size, num_layers, heads, device, forward_expansion, dropout, num_classes)
model.to(device)

CustomBERT(
  (layers): ModuleList(
    (0-7): 8 x TransformerBlock(
      (attention): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=1186, out_features=1186, bias=True)
      )
      (norm1): LayerNorm((1186,), eps=1e-05, elementwise_affine=True)
      (norm2): LayerNorm((1186,), eps=1e-05, elementwise_affine=True)
      (feed_forward): Sequential(
        (0): Linear(in_features=1186, out_features=4744, bias=True)
        (1): ReLU()
        (2): Linear(in_features=4744, out_features=1186, bias=True)
      )
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (dropout): Dropout(p=0.1, inplace=False)
  (fc_out): Linear(in_features=1186, out_features=8, bias=True)
  (sig): Sigmoid()
)

In [80]:
import torch.optim as optim
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score

# Hyperparameters
batch_size = 256
learning_rate = 1e-4
num_epochs = 10

# Prepare DataLoader utilities
def batch_loader(X, y, batch_size=32):
    for i in range(0, len(X) - batch_size + 1, batch_size):
        indices = slice(i, i + batch_size)
        yield torch.tensor(X[indices], dtype=torch.float32), torch.tensor(y[indices], dtype=torch.long)

# Optimizer and Loss
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# Training loop
model.train()
for epoch in range(num_epochs):
    epoch_loss = 0
    correct = 0
    total = 0
    
    X_train, y_train = shuffle(X_train, y_train)  # Shuffle the data each epoch
    for data, targets in batch_loader(X_train, y_train, batch_size):
        data, targets = data.to(device), targets.to(device)
        
        # Forward pass
        outputs = model(data, mask=None)  # Assuming no mask is needed
        loss = criterion(outputs, targets)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()
    
    # Print epoch information
    train_acc = 100 * correct / total
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss/total:.4f}, Accuracy: {train_acc:.2f}%')

# Validation loop
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for data, targets in batch_loader(X_test, y_test, batch_size):
        data, targets = data.to(device), targets.to(device)
        outputs = model(data, mask=None)
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()
    
    val_acc = 100 * correct / total
    print(f'Validation Accuracy: {val_acc:.2f}%')


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x256 and 1186x8)